# Unmapped MARC fields and subfields
This notebook:
* Documents MARC fields and subfields omitted from Argot mappings
* Identifies MARC mapping decisions to be made, by analyzing:
 * current MARC specification from http://www.loc.gov/marc/bibliographic/ecbdlist.html 
 * Argot mappings from https://github.com/trln/data-documentation/blob/master/argot/argot.xlsx
 * Our explicit field and subfield omission decisions 

MARC fields we have decided to exclude from Argot mappings are recorded in [`unmapped_marc_tags.json`](https://github.com/trln/data-documentation/blob/master/marc/unmapped_marc_tags.json).

Specific MARC subfields excluded from Argot mappings *when some subfields from the field ARE mapped* are recorded in [`unmapped_marc_subfields.json`](https://github.com/trln/data-documentation/blob/master/marc/unmapped_marc_subfields.json).

## Use the current MARC specification
First, run script to generate JSON hash of current MARC specification. If successful: 
* Out will = true
* `marc_definition.json` is written to `data-documentation/marc` folder

In [63]:
require './extract_marc_bib_spec'

false

## Get all our mappings data
This next bit just reads in all the data we'll need: 
* `marc_spec` is the updated MARC specification created in the previous step
* `unmapped_fields` is our explicit decisions about MARC fields not mapped to Argot
* `unmapped_subfields` is our explicit decisions about individual MARC subfields not mapped to Argot. These subfields all occur in MARC fields that are otherwise mapped to Argot. 

In [92]:
marc_spec = JSON.parse(File.read('marc_definition.json'))
unmapped_fields = JSON.parse(File.read('unmapped_marc_tags.json'))
unmapped_subfields = JSON.parse(File.read('unmapped_marc_subfields.json'))
puts ''

Next get our current MARC-to-Argot mappings from Argot spreadsheet and turn this into a hash we can use:

In [93]:
require 'simple_xlsx_reader'
doc = SimpleXlsxReader.open('../argot/argot.xlsx')
mappings_d = doc.sheets[1].data
mappings_h = doc.sheets[1].headers
marc_only = mappings_d.select{ |r| r[2] == "MARC"}.map{ |r| [r[5].rjust(3,'0'), r[6], r[3]] }
# Remove mappings for fixed fields and those where data is a constant
marc_only.reject!{ |r| r[0] =~ /(00[678]|\{na\})/}
# Change {na}, {varies}, period, space, or parentheses in subfield list to nothing
marc_only.map!{ |r| [r[0], r[1].gsub(/\{(na|varies)\}|[.() ]/, ''), r[2]]}
# Convert subfield list to array
marc_only.map!{ |r| [r[0], r[1].split(''), r[2]]}
# explode mappings list, so there's one array per subfield value
# orig: ["111", ["j", "4"], "n"]
# new: ["111", "j", "n"], ["111", 4", "n"]
flat_mappings = []
marc_only.each do |r|  
  r[1].each { |sf| flat_mappings << [r[0], sf, r[2]] }
end
mapping_hash = {}
flat_mappings.uniq!.each do |r|
  if mapping_hash.has_key?(r[0])
    mapping_hash[r[0]][r[1]] = r[2]
  else
    mapping_hash[r[0]] = {r[1] => r[2]}
  end
end
puts ''

The above creates a hash with the following basic format: 

```
{
 "581": {
    "a": "n",
    "z": "n",
    "3": "n"
  },
  ...
}
```

Subfields a, z, and 3 from the 581 field are mapped to Argot. 

The "n" at the end of each subfield line means these are final mappings for production. "y" in this position indicates a provisional mapping, which may not yet be implemented or which is subject to major changes.

## Unmapped MARC, field tag level

### Explicit decisions and justifications
The block below reports on the fields we've explicitly decided not to map.

In [66]:
unmapped_tag_table = { 'MARC tag' => [],
                       'Name' => [],
                       'Omission type' => [],
                       'Omission category' => [],
                       'Details' => []
                     }
unmapped_fields.sort.each do |tag, info|
  unmapped_tag_table['MARC tag'] << tag
  unmapped_tag_table['Name'] << marc_spec[tag]['name']
  unmapped_tag_table['Omission type'] << info['omission_type']
  unmapped_tag_table['Omission category'] << info['omission_reason_category']
  unmapped_tag_table['Details'] << info['omission_details']
end
IRuby.display IRuby.table(unmapped_tag_table, maxrows: 100)

MARC tag,Name,Omission type,Omission category,Details
306,PLAYING TIME,permanent,data unusable,"Usually duplicates data recorded in 300 and/or 505. Recorded in a cryptic manner and there's no way to specify which duration recorded goes with which part of the described item, if it's a multi-part thing"
307,"HOURS, ETC.",permanent,irrelevant,"No fields in UNC catalog. Hours an item is available is kind of a weird concept in the catalog, too."
336,CONTENT TYPE,temporary,low priority,Terms used are not user-friendly; this information is usually better expressed elsewhere in the record
337,MEDIA TYPE,temporary,low priority,Terms used are not user-friendly; this information is usually better expressed elsewhere in the record
338,CARRIER TYPE,temporary,low priority,Terms used are not user-friendly; this information is usually better expressed elsewhere in the record
342,GEOSPATIAL REFERENCE DATA,temporary,low priority,"Complex data mapping; appears in <2000 records in UNC catalog on 2018-05-21. Need to assess if this adds useful info, and if so, how to map."
343,PLANAR COORDINATE DATA,temporary,infrequently used,only 2 records in UNC catalog
348,FORMAT OF NOTATED MUSIC,temporary,low priority,Look into whether it should map to genre
355,SECURITY CLASSIFICATION CONTROL,permanent,irrelevant,No records in UNC catalog
357,ORIGINATOR DISSEMINATION CONTROL,permanent,irrelevant,No records in UNC catalog


#<CZTop::Socket::PUB:0x1ac82b0 last_endpoint="tcp://127.0.0.1:59847">

### Unmapped MARC fields without recorded decision and/or justification
And the following lists all other **non-obsolete, non-local** unmapped MARC variable fields we haven't made an explicit decision about:

In [95]:
unmapped = {}
marc_spec.each do |tag, info|
  if mapping_hash.has_key?(tag)
    next
  elsif unmapped_fields.has_key?(tag)
    next
  elsif tag =~ /00[356789]/
    next
  elsif info['context'] =~ /OBSOLETE|LOCAL/
  else
    unmapped[tag] = {'name' => info['name'].strip,
                     'context' => info['context']
      }
  end
end

table2 = { 'MARC tag' => [],
           'Description' => [],
           'Note' => []
         }
unmapped.sort.each do |tag, info|
  table2['MARC tag'] << tag
  table2['Description'] << info['name']
  table2['Note'] << info['context']
end
puts ''

Number of unmapped MARC tags:

In [ ]:
puts table2['MARC tag'].size

In [ ]:
IRuby.display IRuby.table(table2, maxrows: 100)

## Unmapped MARC, subfield level
Here, we are looking at subfields in MARC fields that are at least partly mapped to Argot. 
What subfields in such partially mapped fields are **not** mapped to Argot? 

### Explicit subfield exclusion decisions:

In [68]:
unmapped_sf_table = { 'MARC tag' => [],
                      'Subfield code' => [],
                       'Field name' => [],
                       'Subfield name' => [],
                       'Omission type' => [],
                       'Omission category' => [],
                       'Details' => []
                     }
unmapped_subfields.sort.each do |tag, sfs|
  sfs.each do |code, info|
    unmapped_sf_table['MARC tag'] << tag
    unmapped_sf_table['Subfield code'] << code
    unmapped_sf_table['Field name'] << marc_spec[tag]['name']
    unmapped_sf_table['Subfield name'] << marc_spec[tag]['subfields'][code]['name']
    unmapped_sf_table['Omission type'] << info['omission_type']
    unmapped_sf_table['Omission category'] << info['omission_reason_category']
    unmapped_sf_table['Details'] << info['omission_details']
  end
end
IRuby.display IRuby.table(unmapped_sf_table, maxrows: 100)

MARC tag,Subfield code,Field name,Subfield name,Omission type,Omission category,Details
022,m,INTERNATIONAL STANDARD SERIAL NUMBER,Canceled ISSN-L,,,
022,y,INTERNATIONAL STANDARD SERIAL NUMBER,Incorrect ISSN,,,
022,z,INTERNATIONAL STANDARD SERIAL NUMBER,Canceled ISSN,,,
567,b,METHODOLOGY NOTE,Controlled term,temporary,low priority,UNC has no data in this subfield. Consider mapping to genre_headings and subject_genre if ever meaningfully populated


#<CZTop::Socket::PUB:0x1ac82b0 last_endpoint="tcp://127.0.0.1:59847">

### Unmapped subfields from MARC specification
These subfields may have been overlooked. If an explicit decision has been made to exclude this subfield from mapping to Argot, that decision should be documented in https://github.com/trln/data-documentation/blob/master/marc/unmapped_marc_subfields.json

Ignores: 
* Obsolete or local subfields
* Subfield 0 where subfield name = Authority record control number or standard number
* Subfield 1 where subfield name = Real World Object URI
* Subfield 2 where subfield name starts with Source
* Subfield 6 where subfield name = Linkage
* Subfield 8 where subfield name = Field link and sequence number

First we flatten out all subfield data so it's easier to work with:

In [86]:
subfields = []
marc_spec.each do |tag, info|
  if info['subfields'].size > 0
    info['subfields'].each do |code, sfinfo|
      subfields << {'tag' => tag,
        'code' => code,
        'name' => "#{info['name']} - #{sfinfo['name']}",
        'context' => sfinfo['context']}      
    end
  end
end

puts subfields.size

2491


Then, get rid of subfields that are in unmapped MARC fields. Keep only those whose field tag appears in our mappings spreadsheet.

In [87]:
subfields.select!{ |sf| mapping_hash.has_key?(sf['tag']) }
puts subfields.size

1491


Then, remove:
* Obsolete or local subfields
* Subfield 0 where subfield name = Authority record control number or standard number
* Subfield 1 where subfield name = Real World Object URI
* Subfield 2 where subfield name starts with Source
* Subfield 6 where subfield name = Linkage
* Subfield 7 where subfield name = Control subfield
* Subfield 8 where subfield name = Field link and sequence number

In [88]:
subfields.reject!{ |sf| sf['code'] == '0' && sf['name'] =~ / - Authority record control number/  }
subfields.reject!{ |sf| sf['code'] == '1' && sf['name'] =~ / - Real World Object URI/ }
subfields.reject!{ |sf| sf['code'] == '2' && sf['name'] =~ / - Source/ }
subfields.reject!{ |sf| sf['code'] == '6' && sf['name'] =~ / - Linkage/ }
subfields.reject!{ |sf| sf['code'] == '7' && sf['name'] =~ / - Control subfield/ }
subfields.reject!{ |sf| sf['code'] == '8' && sf['name'] =~ / - Field link and sequence number/ }
subfields.reject!{ |sf| sf['context'] =~ /OBSOLETE|LOCAL/ }
puts subfields.size

1102


Then, remove subfields we have mapped in Argot spreadsheet:

In [89]:
subfields.reject!{ |sf| mapping_hash.has_key?(sf['tag']) && mapping_hash[sf['tag']].has_key?(sf['code']) }
puts subfields.size

150


Then, output the remaining unmapped subfields to a table for analysis:

In [90]:
table4 = { 'MARC tag' => [],
  'subfield code' => [],
  'name' => []
  }

subfields.each do |sf|
  table4['MARC tag'] << sf['tag']
  table4['subfield code'] << sf['code']
  table4['name'] << sf['name']
end
IRuby.display IRuby.table(table4, maxrows: 200)

MARC tag,subfield code,name
015,z,NATIONAL BIBLIOGRAPHY NUMBER - Canceled/Invalid national bibliography number
020,c,INTERNATIONAL STANDARD BOOK NUMBER - Terms of availability
022,m,INTERNATIONAL STANDARD SERIAL NUMBER - Canceled ISSN-L
022,y,INTERNATIONAL STANDARD SERIAL NUMBER - Incorrect ISSN
022,z,INTERNATIONAL STANDARD SERIAL NUMBER - Canceled ISSN
024,c,OTHER STANDARD IDENTIFIER - Terms of availability
028,a,PUBLISHER NUMBER OR DISTRIBUTOR NUMBER - Publisher or distributor number
035,z,SYSTEM CONTROL NUMBER - Canceled/invalid control number
041,b,LANGUAGE CODE - Language code of summary or abstract
041,f,LANGUAGE CODE - Language code of table of contents


#<CZTop::Socket::PUB:0x1ac82b0 last_endpoint="tcp://127.0.0.1:59847">